In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
With SwiftPM flags: []
Working in: /tmp/tmp462ghhc0/swift-install
/home/sgugger/swift/usr/bin/swift: /home/sgugger/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sgugger/swift/usr/bin/swift)
Fetching https://github.com/mxcl/Path.swift
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.3
/home/sgugger/swift/usr/bin/swift: /home/sgugger/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sgugger/swift/usr/bin/swift)
/home/sgugger/swift/usr/bin/swiftc: /home/sgugger/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sgugger/swift/usr/bin/swiftc)
/home/sgugger/swift/usr/bin/swiftc: /home/sgugger/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sgugger/swift/usr/bin/swiftc)[1/10] Compiling Path Path+StringConvertibles.swift
/home/sg

In [2]:
import Foundation
import Path

In [14]:
let path = Path.cwd.parent
print(path)

/home/sgugger/git/harebrain


In [3]:
public extension String {
    func findFirst(pat: String) -> Range<String.Index>? {
        return range(of: pat, options: .regularExpression)
    }
    func hasMatch(pat: String) -> Bool {
        return findFirst(pat:pat) != nil
    }
    func withMaj() -> String {
        return prefix(1).capitalized + dropFirst()
    }
}

In [4]:
public func nbNameToScriptName(_ name: String) -> String {
    var splits = name.components(separatedBy: "_")
    splits = splits[1...].map { $0.withMaj() }
    return splits.joined(separator: "")
}

In [5]:
public func notebookToScript(_ fname: Path, dest: Path?=nil) {
    let newName = nbNameToScriptName(fname.basename(dropExtension: true))+".swift"
    let destFname = (dest ?? fname.parent) / newName
    do {
        let data = try Data(contentsOf: fname.url)
        let jsonData = try JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
This file was autogenerated from \(fname.basename())
        
If you edit it, be sure that:
  1. there is no diff between this file and the corresponding notebook prior to editing
  2. you don't touch the comments looking like // cell ## as it would break the way back to the notebook
        
Run *** when you are done to update the notebooks with your change.
*/
        
"""
        for (i,cell) in cells.enumerated() {
            if let source = cell["source"] as? [String], !source.isEmpty, 
                   source[0].hasMatch(pat: #"^\s*//\s*export\s*$"#) {
                module.append("\n//cell\(i)\n\(source[1...].joined())\n")
            }
        }
        try module.write(to: destFname, encoding: .utf8)
    } catch {
        print("Can't read the content of \(fname)")
    }
}

In [15]:
let fname = path/"/nbs/stable/00_load_data.ipynb"
let dest = path/"Sources/Harebrain"

In [16]:
notebookToScript(fname, dest: dest)

In [17]:
public func makeLibrary(_ nbFolder: Path, dest: Path?=nil){
    for entry in try! nbFolder.ls()  where entry.kind == Entry.Kind.file && entry.path.basename().hasMatch(pat: #"^\d+[a-z]*_.*ipynb$"#) {
        print("Converting \(entry.path.basename())")
        notebookToScript(entry.path, dest: dest)
    }
}

In [23]:
let nbFolder = path/"/nbs/stable"

In [24]:
makeLibrary(nbFolder, dest:dest)

Converting 02a_why_sqrt5.ipynb
Converting 10_mixup_ls.ipynb
Converting 05_anneal.ipynb
Converting 09_optimizer.ipynb
Converting 04_callbacks.ipynb
Converting 08_data_block.ipynb
Converting 08a_heterogeneous_dictionary.ipynb
Converting 06_cuda.ipynb
Converting 01_matmul.ipynb
Converting 02_fully_connected.ipynb
Converting 01a_fastai_layers.ipynb
Converting 11_imagenette.ipynb
Converting 05b_early_stopping.ipynb
Converting 00_load_data.ipynb
Converting 03_minibatch_training.ipynb
Converting 07_batchnorm.ipynb


In [25]:
let fname = dest/"LoadData.swift"

In [26]:
public func scriptToNotebook(_ fname: Path, nbFolder: Path){
    let data = try! Data(contentsOf: fname.url)
    let code: String = String(data: data, encoding: .utf8)!
    let codeCells = code.components(separatedBy: "//cell")
    let nbName = codeCells[0].components(separatedBy: "\n")[1].components(separatedBy: "from ")[1]
    let nbFname = nbFolder / nbName
    let nbData = try! Data(contentsOf: nbFname.url)
    var jsonData = try! JSONSerialization.jsonObject(with: nbData, options: .allowFragments) as! [String: Any]
    var cells = jsonData["cells"] as! [[String:Any]]
    for c in codeCells[1...] {
        var lines = c.components(separatedBy: "\n")
        let idx: Int = Int(lines[0])!
        var i = lines.count-1
        while lines[i].isEmpty { i -= 1}
        if i > 1 {
            for i in 1...(i-1) { lines[i].append("\n") }
        }
        lines[0] = "// export\n"
        cells[idx]["source"] = Array(lines[...i])
    }
    jsonData["cells"] = cells
    let outData = try! JSONSerialization.data(withJSONObject: jsonData, options: .prettyPrinted)  
    let jsonString = String(data: outData, encoding: .utf8)! 
    do { try jsonString.write(to: nbFname, encoding: .utf8) }
    catch { "Couldn't save notebook" }
}

In [27]:
scriptToNotebook(fname, nbFolder: nbFolder)

In [28]:
public func updateNotebooks(_ scriptsFolder: Path, nbFolder: Path) {
    for entry in try! scriptsFolder.ls()  where entry.kind == Entry.Kind.file && entry.path.basename().hasMatch(pat: #".swift$"#) {
        print("Updating nb from \(entry.path.basename())")
        scriptToNotebook(entry.path, nbFolder: nbFolder)
    }
}

In [29]:
updateNotebooks(dest, nbFolder: nbFolder)

Updating nb from WhySqrt5.swift
Updating nb from Anneal.swift
Updating nb from Callbacks.swift
Updating nb from Optimizer.swift
Updating nb from MixupLs.swift
Updating nb from DataBlock.swift
Updating nb from FastaiLayers.swift
Updating nb from FullyConnected.swift
Updating nb from HeterogeneousDictionary.swift
Updating nb from Batchnorm.swift
Updating nb from MinibatchTraining.swift
Updating nb from LoadData.swift
Updating nb from Matmul.swift
Updating nb from Imagenette.swift
Updating nb from EarlyStopping.swift
Updating nb from Cuda.swift
